# `jupyterlite 0.1.0a5+main` vs `pyodide dev`

> This is very much a work-in-progress. See the [Author's log below](#Author's-Log) or follow along on the [draft PR](https://github.com/jupyterlite/jupyterlite/pull/274).

In [1]:
import sys, os
from pathlib import Path
print(len(sys.modules), "are loaded")

932 are loaded


We should now be running python 3.9

In [2]:
assert sys.version.startswith('3.9')
sys.version

'3.9.5 (default, Jul 19 2021, 12:35:25) \n[Clang 13.0.0 (/b/s/w/ir/cache/git/chromium.googlesource.com-external-github.co'

Our "user" should have a writeable `$HOME` dir:

In [3]:
HOME = Path(os.path.expanduser("~"))
HOME

PosixPath('/home/web_user')

In [4]:
assert str(HOME) == os.environ.get("HOME"), f"there's no place like {HOME}"
print(sorted(HOME.glob("*")))

[PosixPath('/home/web_user/.ipython'), PosixPath('/home/web_user/.matplotlib')]


The process launches in `/`

In [5]:
os.getcwd()

'/'

In [6]:
assert os.getcwd() == "/"

## JS

In [7]:
import pyodide_js, js
print(len(sys.modules))

932


### FS: The Emscripten Filesystem API

We're now expecting the newly-hoisted `pyodide.FS` API, _with_ all the filesystems, e.g. `IDBFS`

In [8]:
FS = pyodide_js.FS
js.Object.keys(FS)

root,mounts,devices,streams,nextInode,nameTable,currentPath,initialized,ignorePermissions,trackingDelegate,tracking,ErrnoError,genericErrors,filesystems,syncFSRequests,lookupPath,getPath,hashName,hashAddNode,hashRemoveNode,lookupNode,createNode,destroyNode,isRoot,isMountpoint,isFile,isDir,isLink,isChrdev,isBlkdev,isFIFO,isSocket,flagModes,modeStringToFlags,flagsToPermissionString,nodePermissions,mayLookup,mayCreate,mayDelete,mayOpen,MAX_OPEN_FDS,nextfd,getStream,createStream,closeStream,chrdev_stream_ops,major,minor,makedev,registerDevice,getDevice,getMounts,syncfs,mount,unmount,lookup,mknod,create,mkdir,mkdirTree,mkdev,symlink,rename,rmdir,readdir,unlink,readlink,stat,lstat,chmod,lchmod,fchmod,chown,lchown,fchown,truncate,ftruncate,utime,open,close,isClosed,llseek,read,write,allocate,mmap,msync,munmap,ioctl,readFile,writeFile,cwd,chdir,createDefaultDirectories,createDefaultDevices,createSpecialDirectories,createStandardStreams,ensureErrnoError,staticInit,init,quit,getMode,findObject,a

In [9]:
assert FS, "this is not the API you are looking for"

#### POSIX Paths

##### Creating Directories

We can still synchronously `mkdir`:

In [10]:
FOO = Path("/tmp/foo")
FS.mkdir(str(FOO))

[object Object]

In [11]:
assert FOO.exists()
sorted(FOO.glob("*"))

[]

##### Creating Files

We can also synchronously write files:

In [12]:
BAR = FOO / "bar.txt"
FS.writeFile(str(BAR), "hello world") 

In [13]:
assert BAR.read_text() == "hello world"

#### filesystems: implementation-specific storage options

In [14]:
js.Object.keys(FS.filesystems)

MEMFS,IDBFS,NODEFS,WORKERFS,PROXYFS

##### Example: IDBFS

Of interest, as we already make use of `IndexedDB`, is the `FS.filesystems.IDBFS`, which implements bi-directional, asynchronous syncing. User code can make use of this API directly, with some caveats.

###### Creating the mount

In [15]:
WORK = HOME / "work"
FS.mkdir(str(WORK))

[object Object]

In [16]:
FS.mount(FS.filesystems.IDBFS, {}, str(WORK))

[object Object]

## Author's Log

### 2021-07-16
- [#1692](https://github.com/pyodide/pyodide/pull/1692) [`60acd00`](https://github.com/pyodide/pyodide/commit/60acd0089fdba03777e31dc05cad268271666423) merged, exposing `FS`.

### 2021-07-16
- [#1596](https://github.com/pyodide/pyodide/pull/1596) [`f13f33`](https://github.com/pyodide/pyodide/commit/f13f3340e0458bb78ff653383352b61a57cb6c17) merged, ensuring all `filesystem`s are supported

### 2021-07-18 
- `FS` available from `jsdeliver` nightly, but `Pygments.js` failed to load

### 2021-07-19 
- `FS` available from `jsdelivr`
- everything mostly works 🎉
- some proxy borrow errors if left unattended for a while, but no showstoppers for debugging and interactive authoring
  - <details><code>Uncaught (in promise) Error: This borrowed attribute proxy was automatically destroyed in the process of destroying the proxy it was borrowed from. Try using the 'copy' method.</code></details>